# Project - Airline AI Assistant with Gemini

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline using Google's Gemini

## Installation Required

Make sure you have the Google Generative AI package installed:

In [ ]:
# Install Google Generative AI package
!pip install -q -U google-genai

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from google import genai
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
    client = genai.Client(api_key=google_api_key)
else:
    print("Google API Key not set")
    
MODEL = "gemini-2.0-flash-exp"

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# Simple chat function without tools first

def chat(message, history):
    # Convert Gradio format to Gemini format
    gemini_history = []
    for msg in history:
        role = "model" if msg["role"] == "assistant" else "user"
        gemini_history.append({
            "role": role,
            "parts": [{"text": msg["content"]}]
        })
    
    gemini_history.append({
        "role": "user",
        "parts": [{"text": message}]
    })
    
    response = client.models.generate_content(
        model=MODEL,
        contents=gemini_history,
        config={
            "system_instruction": system_message
        }
    )
    return response.text

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by frontier LLMs.

With tools (also called Function Calling), you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city: str) -> str:
    """
    Get the price of a return ticket to the destination city.
    
    Args:
        destination_city: The city that the customer wants to travel to
    
    Returns:
        The price of the ticket as a string
    """
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("Berlin")

## Gemini Function Calling

Gemini has excellent support for function calling. We need to:
1. Define our function (done above)
2. Declare the function to Gemini when making a request
3. Handle function call requests from Gemini
4. Send the function results back to Gemini

The newer Gemini SDK makes this easier!

In [ ]:
# Gemini can automatically generate function declarations from Python functions!
# We just need to pass the function itself

def chat_with_tools(message, history):
    # Convert history
    gemini_history = []
    for msg in history:
        role = "model" if msg["role"] == "assistant" else "user"
        gemini_history.append({
            "role": role,
            "parts": [{"text": msg["content"]}]
        })
    
    gemini_history.append({
        "role": "user",
        "parts": [{"text": message}]
    })
    
    # First request with tool declaration
    response = client.models.generate_content(
        model=MODEL,
        contents=gemini_history,
        config={
            "system_instruction": system_message,
            "tools": [get_ticket_price]  # Just pass the function!
        }
    )
    
    # Check if Gemini wants to call a function
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        
        # Extract the function name and arguments
        function_name = function_call.name
        function_args = dict(function_call.args)
        
        print(f"Gemini wants to call: {function_name}({function_args})")
        
        # Execute the function
        if function_name == "get_ticket_price":
            result = get_ticket_price(**function_args)
            
            # Add the function call and result to history
            gemini_history.append({
                "role": "model",
                "parts": [{"function_call": function_call}]
            })
            gemini_history.append({
                "role": "user",
                "parts": [{
                    "function_response": {
                        "name": function_name,
                        "response": {"price": result}
                    }
                }]
            })
            
            # Get final response from Gemini
            final_response = client.models.generate_content(
                model=MODEL,
                contents=gemini_history,
                config={
                    "system_instruction": system_message,
                    "tools": [get_ticket_price]
                }
            )
            return final_response.text
    
    return response.text

In [ ]:
gr.ChatInterface(fn=chat_with_tools, type="messages").launch()

## Try asking:
- "How much is a ticket to London?"
- "What's the price for Paris?"
- "I want to fly to Tokyo"

Notice how Gemini automatically calls the function when needed!

## Exercises and Business Applications

Add in more tools - perhaps to simulate actually booking a flight. Consider:
- A `book_flight` function
- A `check_flight_status` function
- A `cancel_booking` function

Next: take this and apply it to your business. Make an AI assistant with tools that could carry out activities for your work. A customer support assistant? New employee onboarding assistant? So many possibilities!